In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

## Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
# create experiment
# client.create_experiment(name="my-test-experiment")

In [4]:
# Runs

from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 2af7f410858a4425b7290525fdf8071a, rmse: 6.2911
run id: 20a63ab597c64e429f4c8d67cfc6ce1d, rmse: 6.2911
run id: aabaccfb97454e3d9d1cdc8054ea958e, rmse: 6.2994
run id: c120784dc360464b8750415ac70bf599, rmse: 6.3024
run id: 7369e17d6404497d8d8366817c0b313e, rmse: 6.3037


## Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

1. Register a new version for the experiment nyc-taxi-regressor
2. Retrieve the latests versions of the model nyc-taxi-regressor and check that a new version 4 was created.
3. Transition the version 1 to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
# model registry
run_id = "2af7f410858a4425b7290525fdf8071a"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Successfully registered model 'nyc-taxi-regressor'.
2022/06/18 00:47:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1655513243081, current_stage='None', description=None, last_updated_timestamp=1655513243081, name='nyc-taxi-regressor', run_id='2af7f410858a4425b7290525fdf8071a', run_link=None, source='./mlruns/1/2af7f410858a4425b7290525fdf8071a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [8]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [9]:
# Staging
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1655513243081, current_stage='Staging', description=None, last_updated_timestamp=1655513248918, name='nyc-taxi-regressor', run_id='2af7f410858a4425b7290525fdf8071a', run_link=None, source='./mlruns/1/2af7f410858a4425b7290525fdf8071a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [10]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1655513243081, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-06-18', last_updated_timestamp=1655513250994, name='nyc-taxi-regressor', run_id='2af7f410858a4425b7290525fdf8071a', run_link=None, source='./mlruns/1/2af7f410858a4425b7290525fdf8071a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

## Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [11]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    #df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    #df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
# download preprocessor
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/luisfucros/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [13]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [14]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [15]:
X_train = preprocess(df_train, dv)
X_val = preprocess(df_val, dv)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

# Autolog parameters
mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2022/06/18 01:08:52 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/luisfucros/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [18]:
# test data
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [19]:
X_test = preprocess(df, dv)

In [20]:
y_test = df[target].values

In [21]:
client.list_registered_models()

[<RegisteredModel: creation_timestamp=1655513243066, description=None, last_updated_timestamp=1655514920767, latest_versions=[<ModelVersion: creation_timestamp=1655513243081, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-06-18', last_updated_timestamp=1655513250994, name='nyc-taxi-regressor', run_id='2af7f410858a4425b7290525fdf8071a', run_link=None, source='./mlruns/1/2af7f410858a4425b7290525fdf8071a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
  <ModelVersion: creation_timestamp=1655514920767, current_stage='None', description='', last_updated_timestamp=1655514920767, name='nyc-taxi-regressor', run_id='646d6363bd3c4eb3b2434db5ebc2828e', run_link='', source='./mlruns/1/646d6363bd3c4eb3b2434db5ebc2828e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [22]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None
version: 3, stage: Production


In [23]:
# test model 
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 122 ms


{'rmse': 6.659623830022513}

In [25]:
%time test_model(name=model_name, stage="None", X_test=X_test, y_test=y_test)

CPU times: user 8.6 s, sys: 456 ms, total: 9.05 s
Wall time: 9.06 s


{'rmse': 6.880732619297816}

In [26]:
# production
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1655513243081, current_stage='Production', description='The model version 1 was transitioned to Staging on 2022-06-18', last_updated_timestamp=1655515770933, name='nyc-taxi-regressor', run_id='2af7f410858a4425b7290525fdf8071a', run_link=None, source='./mlruns/1/2af7f410858a4425b7290525fdf8071a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>